## I. 패키지 밎 자료 불러오기

In [ ]:
!pip install pymysql
!pip install sqlalchemy
!pip install mysqlclient

     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 87 kB 2.8 MB/s 
  Created wheel for mysqlclient: filename=mysqlclient-2.1.0-cp37-cp37m-linux_x86_64.whl size=99976 sha256=980c2494b0bd884b0663896f5ae4dfad6833b10d85e4bfc5450111acdda9b1ba
  Stored in directory: /root/.cache/pip/wheels/97/d4/df/08cd6e1fa4a8691b268ab254bd0fa589827ab5b65638c010b4
Successfully built mysqlclient


In [ ]:
!pip install pyecharts
!pip install chart_studio

     |████████████████████████████████| 135 kB 5.2 MB/s 
     |████████████████████████████████| 130 kB 47.2 MB/s 
     |████████████████████████████████| 64 kB 1.8 MB/s 


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import time
import datetime
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import pymysql
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc

import itertools
import operator

import base64
from io import BytesIO

from datetime import datetime, timedelta

from pyecharts.charts import Bar
from pyecharts.charts import Line
import plotly.offline as plyo
import cufflinks
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
db = pymysql.connect(host=, charset='utf8')
cursor = db.cursor(pymysql.cursors.DictCursor)
sql = 'SELECT * FROM xrp'
cursor.execute(sql)
result = cursor.fetchall()
DF = pd.DataFrame(result)


In [ ]:
DF.tail()

,index,date,coin_name,s,s.1,lang
1102,1102,2021-11-29,xrp,0,8,kor
1103,1103,2021-11-29,xrp,1,3,kor
1104,1104,2021-11-30,xrp,0,4,kor
1105,1105,2021-11-30,xrp,1,3,kor
1106,1106,2021-11-30,xrp,2,2,kor


In [ ]:
# 저장 포인트
DF.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/XRP_sentiment_result.csv')

In [ ]:
#영어를 위해서
db = pymysql.connect(host=, charset='utf8')
cursor = db.cursor(pymysql.cursors.DictCursor)
sql = 'SELECT * FROM cj'
cursor.execute(sql)
result = cursor.fetchall()
DF = pd.DataFrame(result)

### 2) 날짜 자동 정의 함수

In [ ]:
def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates

In [ ]:
date = date_range("2017-01-01", "2021-11-30")

## II. 일단 긍정/중립/부정 3개의 데이터 프래임으로 모으기
- 없을시 0으로 해야 함으로 데이터 프레임을 먼저 만들고 거기에 합치기
- s.1은 갯수

In [ ]:
# date 자동화 for문 작업은 나중에 -> DF['date']의 원소를 리스트 형식으로 받아 date으로 정의
# date = ['2021-11-17','2021-11-18','2021-11-19','2021-11-20','2021-11-21','2021-11-22',
#         '2021-11-23','2021-11-24','2021-11-25','2021-11-26','2021-11-27','2021-11-28','2021-11-29','2021-11-30']
        
# lang = ['kor','eng']
crpyto_name = ['xrp']
s = [0,1,2]

### 1) 새로운 데이터 프레임을 만들어야한다.

In [ ]:
# 새로운 데이터 프레임 만들기
df = pd.DataFrame(columns = ['date', 'crypto_name', 'positive_E','neutrual_E','negative_E','positive_K','neutrual_K','negative_K'])
for i in date:
  for j in crpyto_name:
    df = df.append(pd.DataFrame([[i, j]], columns = ['date','crypto_name']), ignore_index=True)
df

,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K
0,2017-01-01,xrp,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-02,xrp,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-03,xrp,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-04,xrp,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-05,xrp,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1790,2021-11-26,xrp,NaN,NaN,NaN,NaN,NaN,NaN
1791,2021-11-27,xrp,NaN,NaN,NaN,NaN,NaN,NaN
1792,2021-11-28,xrp,NaN,NaN,NaN,NaN,NaN,NaN
1793,2021-11-29,xrp,NaN,NaN,NaN,NaN,NaN,NaN


### 2) 새로운 데이터 프레임에 값들 집어 넣기 - 모든 코인 모든 날짜

In [ ]:
for k in crpyto_name:

  for j in date:

    for i in range(len(DF)):


      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
        if (DF['s'][i] == 0):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
          df['positive_K'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
        if (DF['s'][i] == 0):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
          df['positive_E'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
        if (DF['s'][i] == 1):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
          df['neutrual_K'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
        if (DF['s'][i] == 1):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
          df['neutrual_E'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
        if (DF['s'][i] == 2):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
          df['negative_K'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
        if (DF['s'][i] == 2):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
          df['negative_E'][a] = DF['s.1'][i]    

------ 해킹당한 날 빽업 파일 붙여넣기

In [ ]:
db = pymysql.connect(host=, charset='utf8')
cursor = db.cursor(pymysql.cursors.DictCursor)
sql = 'SELECT * FROM graphing_backup_Nov23'
cursor.execute(sql)
result = cursor.fetchall()
DF_backup = pd.DataFrame(result)


In [ ]:
for k in crpyto_name:

  for j in date:

    for i in range(len(DF_backup)):


      if (DF_backup['date'][i] == j) and (DF_backup['crypto_name'][i] == k):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)].tolist()

          df['positive_E'][a] = DF_backup['positive_E'][i]
          df['neutrual_E'][a] = DF_backup['neutrual_E'][i]
          df['negative_E'][a] = DF_backup['negative_E'][i]
          df['positive_K'][a] = DF_backup['positive_K'][i]
          df['neutrual_K'][a] = DF_backup['neutrual_K'][i]
          df['negative_K'][a] = DF_backup['negative_K'][i]


----- 여기까지

In [ ]:
df = df.fillna(0)

In [ ]:
df.tail(20)

,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K
1775,2021-11-11,xrp,NaN,NaN,NaN,18,2,11
1776,2021-11-12,xrp,NaN,NaN,NaN,9,13,15
1777,2021-11-13,xrp,NaN,NaN,NaN,17,5,11
1778,2021-11-14,xrp,NaN,NaN,NaN,14,2,14
1779,2021-11-15,xrp,NaN,NaN,NaN,15,7,3
1780,2021-11-16,xrp,NaN,NaN,NaN,22,5,12
1781,2021-11-17,xrp,1593,3,4019,13,2,10
1782,2021-11-18,xrp,NaN,NaN,NaN,15,3,11
1783,2021-11-19,xrp,1969,6,4851,17,2,6
1784,2021-11-20,xrp,NaN,NaN,NaN,10,2,6


## --------- 전처리 작업 완료

In [ ]:
# 저장포인트
df.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/XRP_sentiment_result_graphing.csv')

In [ ]:
# 저장포인트
daily_engine = create_engine("mysql+mysqldb://", encoding='utf-8',pool_size = 100000,max_overflow = 0)
daily_conn = daily_engine.connect()
df.to_sql(name='XRP_only',con=daily_conn,if_exists='replace')
print('업데이트 완료 : XRP_only', '\n' )

업데이트 완료 : XRP_only 



In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/XRP_sentiment_result_graphing.csv')

## --------- DB와 코렙에 저장 포인트

## III. 지수화
- 일단 긍정의 전체비율만을 가지고 한다. (대부분의 경우 상승에 관심 있고 롱 포지션을 가지기때문에)
- 영어가 대부분이고 영어는 거의 중립이 없어서 가능할지도
- 나중에 부정의 비율도 고려한다.


### 1) 긍정비율

In [ ]:
df['positive_ratio_E'] = df['positive_E']/(df['positive_E']+df['neutrual_E']+df['negative_E'])
df['positive_ratio_K'] = df['positive_K']/(df['positive_K']+df['neutrual_K']+df['negative_K'])

In [ ]:
df.head(30)

,Unnamed: 0,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
0,0,2017-01-01,xrp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2017-01-02,xrp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2017-01-03,xrp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2017-01-04,xrp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2017-01-05,xrp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,2017-01-06,xrp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,2017-01-07,xrp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,2017-01-08,xrp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,2017-01-09,xrp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,2017-01-10,xrp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.tail(30)

,Unnamed: 0,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
1765,1765,2021-11-01,xrp,NaN,NaN,NaN,14.0,5.0,5.0,NaN,0.583333
1766,1766,2021-11-02,xrp,NaN,NaN,NaN,15.0,8.0,5.0,NaN,0.535714
1767,1767,2021-11-03,xrp,NaN,NaN,NaN,8.0,5.0,5.0,NaN,0.444444
1768,1768,2021-11-04,xrp,NaN,NaN,NaN,13.0,5.0,6.0,NaN,0.541667
1769,1769,2021-11-05,xrp,NaN,NaN,NaN,14.0,6.0,3.0,NaN,0.608696
1770,1770,2021-11-06,xrp,NaN,NaN,NaN,7.0,5.0,4.0,NaN,0.437500
1771,1771,2021-11-07,xrp,NaN,NaN,NaN,2.0,4.0,9.0,NaN,0.133333
1772,1772,2021-11-08,xrp,NaN,NaN,NaN,14.0,4.0,6.0,NaN,0.583333
1773,1773,2021-11-09,xrp,NaN,NaN,NaN,12.0,5.0,5.0,NaN,0.545455
1774,1774,2021-11-10,xrp,NaN,NaN,NaN,11.0,7.0,10.0,NaN,0.392857


## IV. 암호화폐별 추출 밎 가격테이블 연결

### 1) 가격데이터, 감정데이터 불러오기; 각코인별 데이터 프레임 만들기

In [ ]:
crypto_name_p = ['XRP']

for i in crypto_name_p:
    db = pymysql.connect(host=,db='upbit', charset='utf8')
    cursor = db.cursor(pymysql.cursors.DictCursor)
    sql = 'SELECT * FROM {0}'.format(i)
    cursor.execute(sql)
    result = cursor.fetchall()
    globals()[f'{i}'] = pd.DataFrame(result)



crypto_p = [XRP]
crypto_name_p = ['XRP']

for j1,j2,z1 in zip(crypto_p,crypto_p,crypto_name_p):

  X = []

  for i in range(len(j2)):
    date_time_obj = datetime.strftime(j2['index'][i], '%Y-%m-%d')
    a = date_time_obj
    b = j2['close'][i]
    # c = j2['value'][i]
    d = j2['volume'][i]
    e = 0
    f = 0
    g = 0
    h = 0
    i = 0
    j = 0
    k = 0
    l = 0

    x = [a,b,d,e,f,g,h,i,j,k,l]
    X.append(x)
  globals()['{0}'.format(z1)] = pd.DataFrame(X)
  globals()['{0}'.format(z1)].columns = ['date','close_KRW','volume','positive_E','neutrual_E','negative_E',	'positive_K',	'neutrual_K',	'negative_K','positive_ratio_E','positive_ratio_K']

In [ ]:
XRP.tail(20)

,date,close_KRW,volume,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
1506,2021-11-12,1450.0,2.971702e+08,0,0,0,0,0,0,0,0
1507,2021-11-13,1440.0,1.428960e+08,0,0,0,0,0,0,0,0
1508,2021-11-14,1440.0,1.156236e+08,0,0,0,0,0,0,0,0
1509,2021-11-15,1425.0,2.039552e+08,0,0,0,0,0,0,0,0
1510,2021-11-16,1340.0,4.614623e+08,0,0,0,0,0,0,0,0
1511,2021-11-17,1340.0,3.449736e+08,0,0,0,0,0,0,0,0
1512,2021-11-18,1285.0,6.187245e+08,0,0,0,0,0,0,0,0
1513,2021-11-19,1345.0,2.626295e+08,0,0,0,0,0,0,0,0
1514,2021-11-20,1355.0,1.513131e+08,0,0,0,0,0,0,0,0
1515,2021-11-21,1325.0,1.424761e+08,0,0,0,0,0,0,0,0


### 2) 가격데이터에 감정데이터 붙여 넣기
- 가격데이터가 더많이 있어서

In [ ]:
crypto_p = [XRP]
crpyto_name = ['xrp']

for k,l in zip(crypto_p,crpyto_name):

  for j in date:

      a = df.index[(df['date'] == j)&(df['crypto_name'] == l)].tolist()
      b = k.index[(k['date'] == j)].tolist()
      k['positive_E'][b] = df['positive_E'][a]
      k['neutrual_E'][b] = df['neutrual_E'][a]
      k['negative_E'][b] = df['negative_E'][a]
      k['positive_K'][b] = df['positive_K'][a]
      k['neutrual_K'][b] = df['neutrual_K'][a]
      k['negative_K'][b] = df['negative_K'][a]
      k['positive_ratio_E'][b] = df['positive_ratio_E'][a]
      k['positive_ratio_K'][b] = df['positive_ratio_K'][a]

In [ ]:
XRP.tail(20)

,date,close_KRW,volume,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
1506,2021-11-12,1450.0,2.971702e+08,NaN,NaN,NaN,9.0,13.0,15.0,NaN,0.243243
1507,2021-11-13,1440.0,1.428960e+08,NaN,NaN,NaN,17.0,5.0,11.0,NaN,0.515152
1508,2021-11-14,1440.0,1.156236e+08,NaN,NaN,NaN,14.0,2.0,14.0,NaN,0.466667
1509,2021-11-15,1425.0,2.039552e+08,NaN,NaN,NaN,15.0,7.0,3.0,NaN,0.600000
1510,2021-11-16,1340.0,4.614623e+08,NaN,NaN,NaN,22.0,5.0,12.0,NaN,0.564103
1511,2021-11-17,1340.0,3.449736e+08,1593.0,3.0,4019.0,13.0,2.0,10.0,0.283704,0.520000
1512,2021-11-18,1285.0,6.187245e+08,NaN,NaN,NaN,15.0,3.0,11.0,NaN,0.517241
1513,2021-11-19,1345.0,2.626295e+08,1969.0,6.0,4851.0,17.0,2.0,6.0,0.288456,0.680000
1514,2021-11-20,1355.0,1.513131e+08,NaN,NaN,NaN,10.0,2.0,6.0,NaN,0.555556
1515,2021-11-21,1325.0,1.424761e+08,NaN,NaN,NaN,12.0,1.0,8.0,NaN,0.571429


## V. DB에 적제

In [ ]:
# 중간저장

XRP.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/XRP_only.csv')

In [ ]:
# 중간저장 backup
crypto_p = [XRP]

for i in crypto_name_p:
  daily_engine = create_engine("mysql+mysqldb://", encoding='utf-8',pool_size = 100000,max_overflow = 0)
  daily_conn = daily_engine.connect()
  globals()[f'{i}'].to_sql(name=f'{i}_backup_only',con=daily_conn,if_exists='replace')
  print('업데이트 완료 :', f'{i}_backup_only', '\n' )

업데이트 완료 : XRP_backup_only 



In [ ]:
# 중간저장
crypto_p = [XRP]

for i in crypto_name_p:
  daily_engine = create_engine("mysql+mysqldb://", encoding='utf-8',pool_size = 100000,max_overflow = 0)
  daily_conn = daily_engine.connect()
  globals()[f'{i}'].to_sql(name=f'{i}_only',con=daily_conn,if_exists='replace')
  print('업데이트 완료 :', f'{i}_only', '\n' )

업데이트 완료 : XRP_only 



In [ ]:
# 불러오기
XRP = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/XRP.csv')

In [ ]:
XRP

,Unnamed: 0,date,close_KRW,volume,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
0,0,2017-09-25,203.0,2.130000e-02,0,0,0,0,0,0,0.000000,0.000000
1,1,2017-09-26,211.0,5.160000e-02,0,0,0,0,0,0,0.000000,0.000000
2,2,2017-09-27,232.0,5.040000e-02,0,0,0,0,0,0,0.000000,0.000000
3,3,2017-09-28,222.0,4.980000e-02,0,0,0,0,0,0,0.000000,0.000000
4,4,2017-09-29,220.0,5.230000e-02,0,0,0,0,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1519,1519,2021-11-25,1290.0,1.190549e+08,337,0,726,32,11,16,0.317027,0.542373
1520,1520,2021-11-26,1295.0,1.024812e+07,2678,11,4923,6,1,0,0.351813,0.857143
1521,1521,2021-11-27,1215.0,1.644314e+08,3142,14,6397,0,0,0,0.328902,NaN
1522,1522,2021-11-28,1220.0,2.488932e+08,0,0,0,0,0,0,0.000000,0.000000
